## Task
Load `data/dataset.csv` with Polars, inspect schema/shape/missingness/labels, explore text length & most common tokens, then train/evaluate a TF‑IDF + linear classifier to predict whether a text is **human** or **AI**.

In [13]:
import dagshub
dagshub.init(repo_owner='thomas-marquis', repo_name='ml-sandbox', mlflow=True)

Accessing as thomas-marquis

Initialized MLflow to track repo "thomas-marquis/ml-sandbox"

Repository thomas-marquis/ml-sandbox initialized!

In [14]:
import mlflow

In [2]:
import os
from pathlib import Path

import polars as pl

DATA_PATH = Path("data") / "dataset.csv"
assert DATA_PATH.exists(), f"Missing dataset file: {DATA_PATH.resolve()}"

df = pl.read_csv(DATA_PATH, infer_schema_length=10_000)
df.head()

id,label,topic,text,length_chars,length_words,quality_score,sentiment,source_detail,timestamp,plagiarism_score,notes
i64,str,str,str,i64,i64,f64,f64,str,str,f64,str
305,"""human""","""food""","""As someone who follows food, I…",126,22,2.37,0.44,"""author_50""","""2023-12-19 11:21:46""",0.045,"""personal_tone"""
341,"""ai""","""travel""","""Analysis indicates that travel…",141,19,3.44,0.6,"""gpt-5-base""","""2024-04-01 17:43:26""",0.007,""""""
48,"""human""","""education""","""I recently experienced educati…",112,17,3.16,-0.43,"""author_33""","""2025-08-28 07:56:09""",0.163,"""personal_tone"""
68,"""ai""","""sports""","""Analysis indicates that sports…",101,14,3.32,0.44,"""claude-2""","""2023-10-23 14:34:48""",0.122,""""""
480,"""human""","""finance""","""In my experience, finance ofte…",78,14,3.52,-0.31,"""author_87""","""2024-11-28 11:31:37""",0.105,""""""


In [3]:
df.shape, df.schema

((500, 12),
 Schema([('id', Int64),
         ('label', String),
         ('topic', String),
         ('text', String),
         ('length_chars', Int64),
         ('length_words', Int64),
         ('quality_score', Float64),
         ('sentiment', Float64),
         ('source_detail', String),
         ('timestamp', String),
         ('plagiarism_score', Float64),
         ('notes', String)]))

In [4]:
# Basic overview: nulls, uniques (approx), and small sample
overview = pl.DataFrame(
    {
        "column": df.columns,
        "dtype": [str(df.schema[c]) for c in df.columns],
        "null_count": [df.select(pl.col(c).is_null().sum()).item() for c in df.columns],
        "n_unique": [df.select(pl.col(c).n_unique()).item() for c in df.columns],
    }
).sort("null_count", descending=True)

overview

column,dtype,null_count,n_unique
str,str,i64,i64
"""id""","""Int64""",0,500
"""label""","""String""",0,2
"""topic""","""String""",0,10
"""text""","""String""",0,459
"""length_chars""","""Int64""",0,131
…,…,…,…
"""sentiment""","""Float64""",0,143
"""source_detail""","""String""",0,95
"""timestamp""","""String""",0,500


In [5]:
# Try to auto-detect likely text and label columns
import re


def _norm(s: str) -> str:
    return re.sub(r"[^a-z0-9]+", "", s.lower())


cols = df.columns
norm = {_norm(c): c for c in cols}

text_candidates = []
for c in cols:
    dt = df.schema[c]
    if dt in (pl.Utf8, pl.String):
        avg_len = df.select(pl.col(c).cast(pl.Utf8).str.len_chars().mean()).item()
        text_candidates.append((c, float(avg_len) if avg_len is not None else -1.0))

text_candidates_sorted = sorted(text_candidates, key=lambda x: x[1], reverse=True)

label_name_hints = {"label", "target", "class", "y", "source", "author", "isai", "ai", "human"}
label_candidates = [c for c in cols if _norm(c) in label_name_hints or any(h in _norm(c) for h in label_name_hints)]

text_col = text_candidates_sorted[0][0] if text_candidates_sorted else None
label_col = label_candidates[0] if label_candidates else None

text_candidates_sorted[:10], label_candidates, (text_col, label_col)

([('text', 115.152),
  ('timestamp', 19.0),
  ('source_detail', 9.468),
  ('topic', 7.636),
  ('notes', 4.082),
  ('label', 3.494)],
 ['label', 'quality_score', 'source_detail'],
 ('text', 'label'))

In [6]:
# Show value counts for label candidates (helps pick the right label column)
for c in label_candidates[:5]:
    print(f"\n=== {c} ===")
    display(
        df.select(pl.col(c)).with_columns(pl.col(c).cast(pl.Utf8)).group_by(c).len().sort("len", descending=True).head(
            20))


=== label ===


label,len
str,u32
"""ai""",251
"""human""",249



=== quality_score ===


quality_score,len
str,u32
"""2.77""",5
"""3.2""",5
"""3.85""",5
"""4.4""",5
"""4.04""",5
…,…
"""3.18""",4
"""3.54""",4
"""2.88""",4



=== source_detail ===


source_detail,len
str,u32
"""claude-2""",61
"""bloomz-7b""",51
"""gpt-5-base""",50
"""gpt-4o-mini""",46
"""llama-3-small""",43
…,…
"""author_87""",4
"""author_32""",4
"""author_93""",4


In [7]:
# If label_col wasn't found well, fall back to any low-cardinality column as label
if label_col is None:
    low_card = []
    for c in df.columns:
        if df.schema[c] in (pl.Utf8, pl.String, pl.Int64, pl.Int32, pl.Int16, pl.Int8, pl.UInt64, pl.UInt32, pl.UInt16,
                            pl.UInt8, pl.Boolean):
            nu = df.select(pl.col(c).n_unique()).item()
            if nu is not None and 1 < nu <= 20:
                low_card.append((c, int(nu)))
    low_card = sorted(low_card, key=lambda x: x[1])
    low_card[:15]

In [8]:
# Text stats: length distribution and empties for the chosen text_col
assert text_col is not None, "Could not infer a text column."
text_stats = (
    df.select(
        pl.col(text_col).cast(pl.Utf8).alias("text")
    )
    .with_columns(
        pl.col("text").fill_null("").alias("text"),
        pl.col("text").str.len_chars().alias("n_chars"),
        pl.col("text").str.count_matches(r"\S+").alias("n_tokens_approx"),
        (pl.col("text").str.strip_chars().str.len_chars() == 0).alias("is_empty"),
    )
    .select(
        pl.len().alias("n_rows"),
        pl.col("is_empty").sum().alias("n_empty"),
        pl.col("n_chars").mean().alias("mean_chars"),
        pl.col("n_chars").median().alias("median_chars"),
        pl.col("n_chars").quantile(0.95).alias("p95_chars"),
        pl.col("n_tokens_approx").mean().alias("mean_tokens_approx"),
        pl.col("n_tokens_approx").median().alias("median_tokens_approx"),
        pl.col("n_tokens_approx").quantile(0.95).alias("p95_tokens_approx"),
    )
)

text_stats

n_rows,n_empty,mean_chars,median_chars,p95_chars,mean_tokens_approx,median_tokens_approx,p95_tokens_approx
u32,u32,f64,f64,f64,f64,f64,f64
500,0,115.152,103.0,205.0,16.798,14.0,30.0


In [9]:
# Quick token frequency (lowercased, alnum words length>=3), sample-based for speed on big data
sample_n = min(df.height, 50_000)

tokens = (
    df.select(pl.col(text_col).cast(pl.Utf8).fill_null("").alias("text"))
    .head(sample_n)
    .with_columns(
        pl.col("text").str.to_lowercase().str.extract_all(r"[a-z0-9]{3,}").alias("tokens")
    )
    .select(pl.col("tokens").explode().alias("token"))
    .filter(pl.col("token").is_not_null() & (pl.col("token").str.len_chars() > 0))
    .group_by("token")
    .len()
    .sort("len", descending=True)
    .head(30)
)

tokens

token,len
str,u32
"""that""",219
"""the""",194
"""with""",171
"""can""",136
"""this""",132
…,…
"""heuristics""",67
"""optimized""",67
"""simple""",67


In [10]:
# Build TF-IDF classifier (scikit-learn): auto-normalize labels to {human, ai}
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

assert label_col is not None, "Could not infer a label column (set label_col manually in a new cell if needed)."

data_ml = (
    df.select(
        pl.col(text_col).cast(pl.Utf8).alias("text"),
        pl.col(label_col).alias("label_raw"),
    )
    .with_columns(
        pl.col("text").fill_null("").alias("text"),
        pl.col("label_raw").cast(pl.Utf8).str.to_lowercase().fill_null("").alias("label_raw"),
    )
    .filter(pl.col("text").str.strip_chars().str.len_chars() > 0)
)


def normalize_label(s: str) -> str:
    s = (s or "").strip().lower()
    if s in {"ai", "machine", "gpt", "llm", "assistant", "generated", "synthetic", "bot", "chatgpt"}:
        return "ai"
    if s in {"human", "person", "written", "organic", "real"}:
        return "human"
    # numeric-ish / boolean-ish common cases
    if s in {"1", "true", "yes"}:
        return "ai"
    if s in {"0", "false", "no"}:
        return "human"
    # heuristic substring
    if "ai" in s or "gpt" in s or "llm" in s or "machine" in s or "synthetic" in s:
        return "ai"
    if "human" in s:
        return "human"
    return s


labels = data_ml.select(pl.col("label_raw")).to_series().to_list()
labels_norm = [normalize_label(x) for x in labels]

X = data_ml.select(pl.col("text")).to_series().to_list()
y = np.array(labels_norm, dtype=object)

# Keep only binary {human, ai}
mask = np.isin(y, ["human", "ai"])
X = [t for t, m in zip(X, mask) if m]
y = y[mask]

unique, counts = np.unique(y, return_counts=True)
unique, counts

(array(['ai', 'human'], dtype=object), array([251, 249]))

In [16]:
# Train/evaluate
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

clf = Pipeline(
    steps=[
        ("tfidf", TfidfVectorizer(
            lowercase=True,
            strip_accents="unicode",
            ngram_range=(1, 2),
            min_df=2,
            max_df=0.95,
            sublinear_tf=True,
        )),
        ("lr", LogisticRegression(
            max_iter=2000,
            class_weight="balanced",
            n_jobs=None,
        )),
    ]
)

mlflow.autolog()

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred, digits=4))
print("Confusion matrix [human, ai] rows x cols:")
print(confusion_matrix(y_test, y_pred, labels=["human", "ai"]))

if hasattr(clf[-1], "predict_proba"):
    proba = clf.predict_proba(X_test)
    ai_idx = list(clf.classes_).index("ai")
    auc = roc_auc_score((y_test == "ai").astype(int), proba[:, ai_idx])
    print(f"ROC-AUC (ai vs human): {auc:.4f}")

mlflow.sklearn.log_model(
    sk_model=clf,
    name="ai-human-clfr",
    input_example=X_train,
    registered_model_name="ai-human-clfr",
)

2025/12/12 16:02:50 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2025/12/12 16:02:51 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'b2e0b69c8af646e5a19dd033b6e636cf', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2025/12/12 16:02:51 WARNING mlflow.sklearn: Unrecognized dataset type <class 'list'>. Dataset logging skipped.
2025/12/12 16:02:55 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/12/12 16:02:58 WARNING mlflow.sklearn: Unrecognized dataset type <class 'list'>. Dataset logging skipped.


🏃 View run luminous-rat-552 at: https://dagshub.com/thomas-marquis/ml-sandbox.mlflow/#/experiments/0/runs/b2e0b69c8af646e5a19dd033b6e636cf
🧪 View experiment at: https://dagshub.com/thomas-marquis/ml-sandbox.mlflow/#/experiments/0


2025/12/12 16:03:02 WARNING mlflow.sklearn: Unrecognized dataset type <class 'list'>. Dataset logging skipped.
2025/12/12 16:03:02 WARNING mlflow.sklearn: Unrecognized dataset type <class 'list'>. Dataset logging skipped.


              precision    recall  f1-score   support

          ai     1.0000    1.0000    1.0000        50
       human     1.0000    1.0000    1.0000        50

    accuracy                         1.0000       100
   macro avg     1.0000    1.0000    1.0000       100
weighted avg     1.0000    1.0000    1.0000       100

Confusion matrix [human, ai] rows x cols:
[[50  0]
 [ 0 50]]
ROC-AUC (ai vs human): 1.0000


2025/12/12 16:03:02 WARNING mlflow.models.signature: Failed to infer the model signature from the input example. Reason: AttributeError("'int' object has no attribute 'lower'"). To see the full traceback, set the logging level to DEBUG via `logging.getLogger("mlflow").setLevel(logging.DEBUG)`.
2025/12/12 16:03:05 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
Successfully registered model 'ai-human-clfr'.
2025/12/12 16:03:14 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ai-human-clfr, version 1
Created version '1' of model 'ai-human-clfr'.


In [12]:
# Inspect most indicative features for each class (top weighted ngrams)
import numpy as np

tfidf = clf.named_steps["tfidf"]
lr = clf.named_steps["lr"]

feature_names = np.array(tfidf.get_feature_names_out())
coef = lr.coef_[0]
classes = list(lr.classes_)
ai_positive = "ai" if classes[1] == "ai" else classes[0]

# coef corresponds to classes_[1] vs classes_[0] in binary for LogisticRegression
# We'll derive directions based on which class is treated as positive in coef sign:
positive_class = classes[1]
neg_class = classes[0]

top_k = 25
top_pos_idx = np.argsort(coef)[-top_k:][::-1]
top_neg_idx = np.argsort(coef)[:top_k]

print(f"Positive direction => {positive_class}")
print("Top features:")
for i in top_pos_idx:
    print(f"{feature_names[i]:<30} {coef[i]: .4f}")

print(f"\nNegative direction => {neg_class}")
print("Top features:")
for i in top_neg_idx:
    print(f"{feature_names[i]:<30} {coef[i]: .4f}")


Positive direction => human
Top features:
my                              1.9848
to                              1.3932
personal opinion                1.3251
personal                        1.3251
my personal                     1.3251
is that                         1.3251
opinion                         1.3251
opinion on                      1.3251
in                              1.2832
in my                           1.2832
who                             1.1839
who follows                     1.1839
as someone                      1.1839
someone                         1.1839
someone who                     1.1839
follows                         1.1839
believe                         1.1839
day                             1.0928
as                              1.0848
my experience                   0.9591
experience                      0.9591
often leads                     0.9591
often                           0.9591
leads to                        0.9591
leads                 